In [1]:
import cudf
import numpy as np
import pandas as pd
import plotly.express as px
import tools

# init paths to data and models
DATA_PATH = '/data/'

MODEL_NAME = 'wv3_ru'
WORD_MODEL_NAME = 'word_top_wv3_ru'
MODEL_PATH = f'{DATA_PATH}models/{MODEL_NAME}/'
EVENTS_PATH = DATA_PATH + 'events/lem/events.csv'
TMP_PATH = f'{DATA_PATH}tmp/{WORD_MODEL_NAME}/'


'success'

n_clusters = list(range(5, 151))
n_top = list(range(1, 51))

In [ ]:
from collections import defaultdict
import math

idf = defaultdict(int)

df = tools.read_events(EVENTS_PATH)

for ind, row in df.iterrows():
    words = set(row['description'].split())
    for word in words:
        idf[word] += 1

doc_len = len(df)
for word in idf:
    idf[word] = math.log(doc_len / idf[word])  

In [ ]:
import gensim
model = gensim.models.Word2Vec.load(MODEL_PATH + 'mdl')
wv = model.wv
del model

In [ ]:
from collections import Counter


norm = lambda m: m / (((m ** 2).sum(axis=1)) ** (1/2)).reshape(m.shape[0], 1)
!mkdir {TMP_PATH}


for top in n_top:
    is_not_null = []
    X = []
    
    for document in df['description']:
        # counting words from document, which have vector in Word Embedding Model
        tf = Counter(filter(lambda w: w in wv.vocab, document.split()))
        is_not_null.append(len(tf) > 0)
        # exist small possibility that some document don't contain any word from the WEM   
        if len(tf) == 0:
            continue
        # calculating tf-idf 
        # it isn't necessary to divide tf into len(documents) due to normalizing vectors
        words, tf_idf = zip(*[(w, tf[w] * idf[w]) for w in tf])
        # sorting words of the document by tf_idf and slicing top of them
        score2word = sorted(zip(tf_idf, words), reverse=True)[:top]
        # calculating a vector for the document as sum(tf_idf(word) * vec(word)) 
        vec = sum(map(lambda p: p[0] * wv[p[1]], score2word))
        X.append(vec)
    
    X = norm(np.array(X))
    df = df[is_not_null]
    df = tools.calculate_2d(df, X)
    
    path = f'{TMP_PATH}/{top}/'
    !mkdir {path}
    np.save(path + 'X.npy', X)
    df.to_csv(r'' + path + 'df.csv', index=False)

In [ ]:
for top in n_top:
    path = f'{TMP_PATH}{top}/'
    df = tools.read_events(path + 'df.csv')
    X = np.load(path + 'X.npy')
    
    df = tools.k_means_list(n_clusters, df, X)
    df_scores = tools.calc_scores_list(n_clusters, df, df_cross)
    
    df_scores.to_csv(r'' + path + 'scores_km.csv', index=False)
    df.to_csv(r'' + path + 'df_km.csv', index=False)

In [ ]:
df_cross = pd.read_csv('cross_valid_union.csv')
for top in n_top:
    path = f'{TMP_PATH}{top}/'
    df = tools.read_events(path + 'df.csv')
    X = np.load(path + 'X.npy')
    
    df = tools.agglomerative_list(n_clusters, df, X)
    df_scores = tools.calc_scores_list(n_clusters, df, df_cross)
    
    df_scores.to_csv(r'' + path + 'scores_ag.csv', index=False)
    df.to_csv(r'' + path + 'df_ag.csv', index=False)

In [4]:
best = tools.find_best_2d_score(n_top, TMP_PATH, 'scores_ag.csv', 'f1')
print(best)

path = f'{TMP_PATH}{best["n_semantic"]}/'
df = tools.read_events(path + 'df_ag.csv')
scores = pd.read_csv(path + 'scores_ag.csv')

tools.plot_score(scores, 'f1').show()
tools.plot_events(df, best['n_clusters']).show()
centroids = tools.calc_centroids(df, best['n_clusters'])
fig, _ = tools.plot_centroids(centroids, size_max=80, size_text_tags=2)
fig.show()

tools.plot_scores_3d(n_top, n_clusters, TMP_PATH, 'scores_ag.csv', 'f1')

{'name': 'f1', 'value': 0.9102196752626552, 'n_clusters': 11, 'n_semantic': 39}


In [29]:
from importlib import reload
reload(tools)
path = f'{TMP_PATH}{20}/'
df = tools.read_events(path + 'df.csv')
X = np.load(path + 'X.npy')
    
df = tools.k_means_list([20], df, X, n_init=200)
df_scores = tools.calc_scores_list([20], df, df_cross)

In [30]:
df_scores

,n_clusters,precision,recall,f1,rand,tp,tn,fp,fn
0,20,0.954116,0.668242,0.785992,0.949027,707,6461,34,351
